In [9]:
import tensorflow as tf
import torch

# check if GPU is being used
print("GPU Status: ", end = '')
if len(tf.config.list_physical_devices('GPU')) != 0: print("Active")
else: print("Inactive")

GPU Status: Active


In [10]:
# TensorFlow and tf.keras
import numpy as np
from tensorflow import keras
import autokeras as ak

#tf version should be 2.5 or higher
tf.__version__

'2.10.0'

In [11]:
#get data
(train_images, train_labels), (test_images, test_labels) = \
      keras.datasets.fashion_mnist.load_data()

In [12]:
#scale data
train_images = train_images / 255.0
test_images = test_images / 255.0

In [13]:
#reshape inputs for CNN layers
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [14]:
import os
from tensorboard.plugins.hparams import api as hp

# set hparams
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'adagrad', 'rmsprop', 'sgd']))
HP_CONV_UNITS = hp.HParam('conv_units', hp.Discrete([6, 12, 24]))
HP_DENSE_UNITS = hp.HParam('dense_units', hp.Discrete([100, 200, 400]))


In [15]:
def train_model(hparams):

  model = keras.Sequential([
    keras.layers.Conv2D(kernel_size = 3, filters = hparams[HP_CONV_UNITS], use_bias = False, padding = 'same', input_shape = (28, 28, 1)),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('gelu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 24, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 32, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 32, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Flatten(),

    keras.layers.Dense(hparams[HP_DENSE_UNITS], use_bias = False),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation = 'softmax')
  ])

  model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
  _, accuracy = model.evaluate(test_images, test_labels)
  return accuracy


In [16]:
num_conv_units = []
num_dense_units = []
units_accuracy = []

for conv_units in HP_CONV_UNITS.domain.values:
    for dense_units in HP_DENSE_UNITS.domain.values:
        hparams = { 
            HP_CONV_UNITS: conv_units,
            HP_DENSE_UNITS: dense_units,
        }
        accuracy = train_model(hparams)
        num_conv_units.append(conv_units)
        num_dense_units.append(dense_units)
        units_accuracy.append(accuracy)

Epoch 1/10
1875/1875 [==============================] - 16s 7ms/step - loss: 0.3679 - accuracy: 0.8692 - val_loss: 0.2695 - val_accuracy: 0.9028
Epoch 2/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2477 - accuracy: 0.9090 - val_loss: 0.2326 - val_accuracy: 0.9135
Epoch 3/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2011 - accuracy: 0.9263 - val_loss: 0.2156 - val_accuracy: 0.9206
Epoch 4/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1680 - accuracy: 0.9385 - val_loss: 0.2399 - val_accuracy: 0.9148
Epoch 5/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1396 - accuracy: 0.9485 - val_loss: 0.2164 - val_accuracy: 0.9241
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1113 - accuracy: 0.9594 - val_loss: 0.2138 - val_accuracy: 0.9287
Epoch 7/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0873 - accuracy: 0.9672 - val_loss: 0.2324 - val_ac

In [17]:
for i in range (9):
        print("---------------------------------------------------------------------------")
        print("CONV Layer Filters: " + str(num_conv_units[i]))
        print("DENSE Layer Units: " + str(num_dense_units[i]))
        print("Accuracy: " + str(units_accuracy[i]))
        print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
CONV Layer Filters: 6
DENSE Layer Units: 100
Accuracy: 0.9276000261306763
---------------------------------------------------------------------------
---------------------------------------------------------------------------
CONV Layer Filters: 6
DENSE Layer Units: 200
Accuracy: 0.9283000230789185
---------------------------------------------------------------------------
---------------------------------------------------------------------------
CONV Layer Filters: 6
DENSE Layer Units: 400
Accuracy: 0.9283000230789185
---------------------------------------------------------------------------
---------------------------------------------------------------------------
CONV Layer Filters: 12
DENSE Layer Units: 100
Accuracy: 0.9284999966621399
---------------------------------------------------------------------------
---------------------------------------------------------------------------
CONV Layer Filters: